In [42]:
import os
from dotenv import load_dotenv, find_dotenv     

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import getpass

api_key = getpass.getpass()
os.environ["MISTRAL_API_KEY"] = api_key

In [43]:
def load_env():
    _ = load_dotenv(find_dotenv())

def load_mistral_api_key(ret_key=False):
    load_env()
    global api_key
    global dlai_endpoint
    api_key = os.getenv("MISTRAL_API_KEY")
    #print(api_key)
    #print(dlai_endpoint)
    dlai_endpoint = os.getenv("DLAI_MISTRAL_API_ENDPOINT")
    
    global client
    client = MistralClient(api_key=api_key)
    
    if ret_key:
        return api_key, dlai_endpoint

def mistral(user_message, 
            model="mistral-small-latest",
            is_json=False):
    #print("api_key",api_key)
    client = MistralClient(api_key=api_key)
    #print(client)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, 
            messages=messages,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model, 
            messages=messages)

    return chat_response.choices[0].message.content


def get_text_embedding(txt):
    global client
    embeddings_batch_response = client.embeddings(
        model="mistral-embed", 
        input=txt
    )
    return embeddings_batch_response.data[0].embedding

import requests
from bs4 import BeautifulSoup
import re
def get_web_article_text(url, file_save_name=None):

    response = requests.get(url)
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    
    tag = soup.find("div", re.compile("^prose--styled"))
    text = tag.text
    
    if file_save_name:
        f = open(file_save_name, "w")
        f.write(text)
        f.close()
    return text

In [44]:

load_mistral_api_key()

In [46]:
mistral("hello, what can you do?")

'Hello! I can assist with a variety of tasks. Here are some examples:\n\n1. Answer questions: I can provide information on a wide range of topics, from general knowledge to specific queries.\n\n2. Provide explanations: I can explain complex concepts in a simple and easy-to-understand manner.\n\n3. Generate ideas: I can help brainstorm ideas for creative projects, problem-solving, or other tasks.\n\n4. Draft emails or messages: I can help write professional or casual emails or messages based on your instructions.\n\n5. Schedule appointments: I can help manage your calendar and schedule meetings or appointments.\n\n6. Translate text: I can translate text from one language to another.\n\n7. Set reminders: I can help manage your tasks by setting reminders for important events or deadlines.\n\n8. Provide news updates: I can give you updates on current events or news in specific areas of interest.\n\n9. Offer learning resources: I can suggest resources for learning new skills or studying spe

In [47]:
prompt = """
    You are a bank customer service bot. 
    Your task is to assess customer intent and categorize customer 
    inquiry after <<<>>> into one of the following predefined categories:
    
    card arrival
    change pin
    exchange rate
    country support 
    cancel transfer
    charge dispute
    
    If the text doesn't fit into any of the above categories, 
    classify it as:
    customer service
    
    You will only respond with the predefined category. 
    Do not provide explanations or notes. 
    
    ###
    Here are some examples:
    
    Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
    Category: card arrival
    Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
    Category: exchange rate 
    Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
    Category: country support
    Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
    Category: customer service
    ###
    
    <<<
    Inquiry: {inquiry}
    >>>
    Category:
"""

In [48]:
response = mistral(f"Please correct the spelling and grammar of \
this prompt and return a text that is the same prompt,\
with the spelling and grammar fixed: {prompt}")

In [49]:
print(response)

You are a bank customer service bot.
Your task is to assess customer intent and categorize the customer inquiry following the <<<>>> into one of the following predefined categories:

card arrival
change PIN
exchange rate
country support
cancel transfer
charge dispute

If the text doesn't fit into any of the above categories, classify it as:
customer service

You will only respond with the predefined category. Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
Category: card arrival

Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fee

In [50]:
mistral(
    response.format(
        inquiry="I am inquiring about the availability of your cards in the EU"
    )
)

'country support'

## Information Extraction with JSON Mode

In [52]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker. 
"""

In [53]:
prompt = f"""
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema: 

{{
        "age": {{
            "type": "integer"
        }},
        "gender": {{
            "type": "string",
            "enum": ["male", "female", "other"]
        }},
        "diagnosis": {{
            "type": "string",
            "enum": ["migraine", "diabetes", "arthritis", "acne"]
        }},
        "weight": {{
            "type": "integer"
        }},
        "smoking": {{
            "type": "string",
            "enum": ["yes", "no"]
        }}
}}
"""

In [54]:
response = mistral(prompt, is_json=True)
print(response)

{"age": 60, "gender": "male", "diagnosis": "diabetes", "weight": 210, "smoking": "yes"}


## Email Personalization

In [55]:
email = """
Dear mortgage lender, 

What's your 30-year fixed-rate APR, how is it compared to the 15-year 
fixed rate?

Regards,
Anna
"""

In [56]:
prompt = f"""

You are a mortgage lender customer service bot, and your task is to 
create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure 
that your response is clear, concise, and directly addresses the 
customer's question. Address the customer in a friendly and 
professional manner. Sign the email with "Lender Customer Support."   
      
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In [57]:
response = mistral(prompt)
print(response)

Subject: Re: Mortgage Interest Rates Inquiry

Dear Anna,

Thank you for reaching out to us regarding our mortgage rates. I'm happy to provide you with the information you're looking for.

Our 30-year fixed-rate mortgage has an APR of 6.484%. In comparison, our 15-year fixed-rate mortgage has a lower APR of 5.848%. This means that while the interest rate for the 15-year term is lower, the overall cost of borrowing, including fees and charges, is also lower when compared to the 30-year term.

I hope this information helps you make an informed decision about your mortgage options. If you have any further questions or need clarification on anything, please don't hesitate to ask.

Best regards,

Lender Customer Support


## Summarization

In [58]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft. 

What’s new: Mao9istral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year). Microsoft invested $16.3 million in the French startup, and it agreed to distribute Mistral Large on its Azure platform and let Mistral AI use Azure computing infrastructure. Mistral AI makes the new models available to try for free here and to use on its La Plateforme and via custom deployments.

Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed. Like the earlier, open source Mistral 7B and Mixtral 8x7B, they can process 32,000 tokens of input context. 

Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, though falling short of GPT-4. Mistral Small, which is optimized for latency and cost, achieved 72.2 percent on MMLU.
Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.
Microsoft’s investment in Mistral AI is significant but tiny compared to its $13 billion stake in OpenAI and Google and Amazon’s investments in Anthropic, which amount to $2 billion and $4 billion respectively.
Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.
Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta. The French government has touted the company as a home-grown competitor to U.S.-based leaders like OpenAI. France’s representatives in the European Commission argued on Mistral’s behalf to loosen the European Union’s AI Act oversight on powerful AI models. 

Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators. The European Commission, which already was investigating Microsoft’s agreement with OpenAI for potential breaches of antitrust law, plans to investigate the new partnership as well. Members of President Emmanuel Macron’s Renaissance party criticized the deal’s potential to give a U.S. company access to European users’ data. However, other French lawmakers support the relationship.

Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to potential customers around the world. It gives the tech giant greater access to the European market. And it gives Azure customers access to a high-performance model that’s tailored to Europe’s unique regulatory environment.

We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with a leading hyperscaler is a sign of the tremendous processing and distribution power that remains concentrated in the large, U.S.-headquartered cloud companies.
"""

In [60]:
prompt = f"""
You are a commentator. Your task is to write a report on a newsletter. 
When presented with the newsletter, come up with interesting questions to ask,
and answer each question. 
Afterward, combine all the information and write a report in the markdown
format. 

# Newsletter: 
{newsletter}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes 
presented in the newsletter.

## Interesting Questions: 
Generate three distinct and thought-provoking questions that can be 
asked about the content of the newsletter. For each question:
- After "Q: ", describe the problem 
- After "A: ", provide a detailed explanation of the problem addressed 
in the question.
- Enclose the ultimate answer in <>.

## Write a analysis report
Using the summary and the answers to the interesting questions, 
create a comprehensive report in Markdown format. 
"""

In [61]:
response = mistral(prompt)
print(response)

# Summary

European AI startup, Mistral AI, has unveiled two new large language models, Mistral Large and Mistral Small, which have outperformed several models from Anthropic, Google, and Meta on the MMLU benchmark. Microsoft has invested $16.3 million in Mistral AI and will distribute Mistral Large on its Azure platform, while Mistral AI will use Azure's computing infrastructure. The partnership has sparked debate among European lawmakers due to potential data access and antitrust concerns.

# Interesting Questions

## Q: What are the performance metrics of Mistral Large and Mistral Small on the MMLU benchmark?

A: Mistral Large achieved 81.2 percent on the MMLU benchmark, surpassing models like Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, but falling short of GPT-4. Mistral Small, optimized for latency and cost, achieved 72.2 percent on the MMLU benchmark. Both models are fluent in French, German, Spanish, and Italian.

## Q: What are the potential implications 

## Mistral Small

In [62]:
prompt = """
Classify the following email to determine if it is spam or not.
Only respond with the exact text "Spam" or "Not Spam". 

# Email:
🎉 Urgent! You've Won a $1,000,000 Cash Prize! 
💰 To claim your prize, please click on the link below: 
https://bit.ly/claim-your-prize
"""

In [63]:
mistral(prompt, model="mistral-small-latest")

'Spam'

## Mistral Medium

In [64]:
prompt = """
Compose a welcome email for new customers who have just made 
their first purchase with your product. 
Start by expressing your gratitude for their business, 
and then convey your excitement for having them as a customer. 
Include relevant details about their recent order. 
Sign the email with "The Fun Shop Team".

Order details:
- Customer name: Anna
- Product: hat 
- Estimate date of delivery: Feb. 25, 2024
- Return policy: 30 days
"""

In [65]:
response_medium = mistral(prompt, model="mistral-medium-latest")

In [66]:
print(response_medium)

Subject: Welcome to The Fun Shop, Anna! Your Hat is on its Way!

Dear Anna,

We are beyond thrilled to welcome you to The Fun Shop family! We want to express our heartfelt gratitude for your decision to make your first purchase with us. Your support means the world to us!

We hope you're as excited as we are about your new hat! Our team has been working hard to ensure that your order is processed and shipped as quickly as possible. According to our records, your hat is expected to arrive at your doorstep on February 25, 2024. We will keep you updated on the status of your order and will let you know as soon as it's shipped.

We want you to be completely satisfied with your purchase, so if for any reason you're not happy with your hat, please don't hesitate to reach out to us. We offer a 30-day return policy, so you have plenty of time to decide if it's the right fit for you.

Thank you once again for choosing The Fun Shop for your shopping needs. We're committed to providing you with t

## Mistral Large

In [67]:
prompt = """
Calculate the difference in payment dates between the two \
customers whose payment amounts are closest to each other \
in the following dataset. Do not write code.

# dataset: 
'{
  "transaction_id":{"0":"T1001","1":"T1002","2":"T1003","3":"T1004","4":"T1005"},
    "customer_id":{"0":"C001","1":"C002","2":"C003","3":"C002","4":"C001"},
    "payment_amount":{"0":125.5,"1":89.99,"2":120.0,"3":54.3,"4":210.2},
"payment_date":{"0":"2021-10-05","1":"2021-10-06","2":"2021-10-07","3":"2021-10-05","4":"2021-10-08"},
    "payment_status":{"0":"Paid","1":"Unpaid","2":"Paid","3":"Paid","4":"Pending"}
}'
"""

In [68]:
response_small = mistral(prompt, model="mistral-small-latest")

In [69]:
print(response_small)

To solve this, we first need to identify the two customers with the closest payment amounts. From the dataset, we can see that C001 and C003 have the closest payment amounts (125.5 and 120.0).

Next, we need to find the difference in payment dates between these two customers. C001 made a payment on 2021-10-05 and C003 made a payment on 2021-10-07.

So, the difference in payment dates between C001 and C003 is 2 days.


In [70]:
response_large = mistral(prompt, model="mistral-large-latest")

In [71]:
print(response_large)

To find the difference in payment dates between the two customers whose payment amounts are closest to each other, we first need to identify the two closest payment amounts.

In the provided dataset, the payment amounts are as follows:

- T1001: $125.50
- T1002: $89.99
- T1003: $120.00
- T1004: $54.30
- T1005: $210.20

The two closest payment amounts are $125.50 and $120.00, with a difference of $5.50. These amounts correspond to transactions T1001 and T1003, made by customers C001 and C003, respectively.

Now, let's find the difference in payment dates:

- T1001 was made on 2021-10-05.
- T1003 was made on 2021-10-07.

To calculate the difference between these two dates:

- Subtract the earlier date (2021-10-05) from the later date (2021-10-07).

The difference in payment dates between the two customers whose payment amounts are closest to each other is 2 days.


## Expense 